In [1]:
import numpy as np
import pandas as pd
import datetime
import gc
import time
import itertools
import matplotlib.pyplot as plt
import seaborn as sns

import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor

from sklearn.model_selection import StratifiedKFold, RepeatedKFold, KFold
from sklearn.metrics import mean_squared_error, roc_auc_score, log_loss
from sklearn.impute  import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import BayesianRidge, LinearRegression

import warnings
warnings.filterwarnings('ignore')

RANDOM_STATE_TSG = 2019
np.random.seed(RANDOM_STATE_TSG)

file_path = 'F:/github_me_repos/Kaggle_code/santander-customer-transaction-prediction/'
print('finished!')

finished!


In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

print('finished!')

In [2]:
df_train = pd.read_csv(file_path + 'data/train.csv')
df_test = pd.read_csv(file_path + 'data/test.csv')

print('finished!')

finished!


In [3]:
df_train.head(10)

ID_code  target    var_0   var_1    var_2   var_3    var_4    var_5  \
0  train_0       0   8.9255 -6.7863  11.9081  5.0930  11.4607  -9.2834   
1  train_1       0  11.5006 -4.1473  13.8588  5.3890  12.3622   7.0433   
2  train_2       0   8.6093 -2.7457  12.0805  7.8928  10.5825  -9.0837   
3  train_3       0  11.0604 -2.1518   8.9522  7.1957  12.5846  -1.8361   
4  train_4       0   9.8369 -1.4834  12.8746  6.6375  12.2772   2.4486   
5  train_5       0  11.4763 -2.3182  12.6080  8.6264  10.9621   3.5609   
6  train_6       0  11.8091 -0.0832   9.3494  4.2916  11.1355  -8.0198   
7  train_7       0  13.5580 -7.9881  13.8776  7.5985   8.6543   0.8310   
8  train_8       0  16.1071  2.4426  13.9307  5.6327   8.8014   6.1630   
9  train_9       0  12.5088  1.9743   8.8960  5.4508  13.6043 -16.2859   

    var_6    var_7   ...     var_190  var_191  var_192  var_193  var_194  \
0  5.1187  18.6266   ...      4.4354   3.9642   3.1364   1.6910  18.5227   
1  5.6208  16.5338   ...      7.6421   7.7214   2.5837  10.9516  15.4305   
2  6.9427  14.6155   ...      2.9057   9.7905   1.6704   1.6858  21.6042   
3  5.8428  14.9250   ...      4.4666   4.7433   0.7178   1.4214  23.0347   
4  5.9405  19.2514   ...     -1.4905   9.5214  -0.1508   9.1942  13.2876   
5  4.5322  15.2255   ...     -6.3068   6.6025   5.2912   0.4403  14.9452   
6  6.1961  12.0771   ...      8.7830   6.4521   3.5325   0.1777  18.3314   
7  5.6890  22.3262   ...     13.1700   6.5491   3.9906   5.8061  23.1407   
8  4.4514  10.1854   ...      1.4298  14.7510   1.6395   1.4181  14.8370   
9  6.0637  16.8410   ...      0.5543   6.3160   1.0371   3.6885  14.8344   

   var_195  var_196  var_197  var_198  var_199  
0  -2.3978   7.8784   8.5635  12.7803  -1.0914  
1   2.0339   8.1267   8.7889  18.3560   1.9518  
2   3.1417  -6.5213   8.2675  14.7222   0.3965  
3  -1.2706  -2.9275  10.2922  17.9697  -8.9996  
4  -1.5121   3.9267   9.5031  17.9974  -8.8104  
5   1.0314  -3.6241   9.7670  12.5809  -4.7602  
6   0.5845   9.1104   9.1143  10.8869  -3.2097  
7  -0.3776   4.2178   9.4237   8.6624   3.4806  
8  -1.9940  -1.0733   8.1975  19.5114   4.8453  
9   0.4467  14.1287   7.9133  16.2375  14.2514  

[10 rows x 202 columns]

In [7]:
df_test.dtypes
non_numerics = [x for x in df_test.columns if not (df_test[x].dtype == np.float64 or df_test[x].dtype == np.int64)]
print('non_numerics is ', non_numerics)

In [10]:
df_train.shape, df_test.shape

((200000, 202), (200000, 201))

In [8]:
train_y = df_train['target']
train_X = df_train.drop(columns=['ID_code', 'target'], inplace=True)

test_id = test_df_test['ID_code']
test_X = df_test.drop(columns=['ID_code'], inplace=True)

['ID_code']

ID_code    var_0    var_1    var_2   var_3    var_4    var_5   var_6  \
0  test_0  11.0656   7.7798  12.9536  9.4292  11.4327  -2.3805  5.8493   
1  test_1   8.5304   1.2543  11.3047  5.1858   9.1974  -4.0117  6.0196   
2  test_2   5.4827 -10.3581  10.1407  7.0479  10.2628   9.8052  4.8950   
3  test_3   8.5374  -1.3222  12.0220  6.5749   8.8458   3.1744  4.9397   
4  test_4  11.7058  -0.1327  14.1295  7.7506   9.1035  -8.5848  6.8595   
5  test_5   5.9862  -2.2913   8.6058  7.0685  14.2465  -8.6761  4.2467   
6  test_6   8.4624  -6.1065   7.3603  8.2627  12.0104  -7.2073  4.1670   
7  test_7  17.3035  -2.4212  13.3989  8.3998  11.0777   9.6449  5.9596   
8  test_8   6.9856   0.8402  13.7161  4.7749   8.6784 -13.7607  4.3386   
9  test_9  10.3811  -6.9348  14.6690  9.0941  11.9058 -10.8018  3.4508   

     var_7   var_8   ...     var_190  var_191  var_192  var_193  var_194  \
0  18.2675  2.1337   ...     -2.1556  11.8495  -1.4300   2.4508  13.7112   
1  18.6316 -4.4131   ...     10.6165   8.8349   0.9403  10.1282  15.5765   
2  20.2537  1.5233   ...     -0.7484  10.9935   1.9803   2.1800  12.9813   
3  20.5660  3.3755   ...      9.5702   9.0766   1.6580   3.5813  15.1874   
4  10.6048  2.9890   ...      4.2259   9.1723   1.2835   3.3778  19.5542   
5  14.7632  1.8790   ...     -2.1115   7.1178  -0.4249   8.8781  14.9438   
6  13.0809 -4.3004   ...     12.3609   6.8661   4.0971   8.8484  17.5010   
7  17.8477 -4.8068   ...      4.4676   4.4214   0.9303   1.4994  15.2648   
8  14.5843  2.5883   ...     -3.4657   7.8754   2.4698  -0.0362  16.7144   
9  20.2816 -1.4112   ...      1.8052  11.0723   0.8907   4.7680  15.1425   

   var_195  var_196  var_197  var_198  var_199  
0   2.4669   4.3654  10.7200  15.4722  -8.7197  
1   0.4773  -1.4852   9.8714  19.1293 -20.9760  
2   2.1281  -7.1086   7.0618  19.8956 -23.1794  
3   3.1656   3.9567   9.2295  13.0168  -4.2108  
4  -0.2860  -5.1612   7.2882  13.9260  -9.1846  
5  -2.2151  -6.0233   9.8117  17.1127  10.8240  
6   0.0295   7.7443   9.1509  18.4736   5.1499  
7  -1.7931   6.5316  10.4855  23.4631   0.7283  
8   0.1221  -1.4328   9.9207  16.9865  -3.3304  
9   0.6075  -4.4447   9.5788  15.8146   9.3457  

[10 rows x 201 columns]